<a href="https://colab.research.google.com/github/eric88525/Tbrain_NLP_game/blob/master/train_QA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Wed Aug  5 14:17:02 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.57       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8     9W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)
!mkdir -p drive
!google-drive-ocamlfuse drive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
/bin/bash: google-drive-ocamlfuse: command not found


In [ ]:
!pip install transformers
!pip install opencc
!pip install pyprind
!pip install zhon


     |████████████████████████████████| 778kB 4.7MB/s 
     |████████████████████████████████| 3.0MB 24.0MB/s 
     |████████████████████████████████| 890kB 38.2MB/s 
     |████████████████████████████████| 1.1MB 51.7MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=cf333b2804b5e4a3d3394fe60e86d42e4e1cef3c28a80ac31c303424e9bbd4e1
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 1.3MB 4.7MB/s 
     |████████████████████████████████| 102kB 3.8MB/s 
  Created wheel for zhon: filename=zhon-1.1.5-cp36-none-any.whl size=84293 sha256=c68b8428e21f2aa483d65fbcec7d24486725d41c7d39e35775483a9a0826754c
  Stored in directory: /root/.cache/pip/wheels/0e/93/5a/ad2f403c359ba996e33c21bf18611d921413df9740ede2fcf4
Successfully built zhon


In [ ]:
import torch
import torch.nn as nn
from torch import optim
from transformers import *
import pandas as pd
import ast
import copy
import os
from time import strftime,gmtime
from opencc import OpenCC
import pyprind
#from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import re
from zhon.hanzi import non_stops,stops
import numpy as np
import random
print(torch.cuda.is_available())
!nvidia-smi

True
Wed Aug  5 14:18:15 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.57       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8     9W /  70W |     10MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+--

In [ ]:
cd ./drive/My Drive/Colab Notebooks/TBrain/nosep/

/content/drive/My Drive/Colab Notebooks/TBrain/nosep


In [ ]:
def eval(pred, ans):
    if bool(pred) is not bool(ans):
        return 0
    elif not pred and not ans:
        return 1
    else:
        pred = set(pred)
        ans = set(ans)
        interaction_len = len(pred & ans)
        if interaction_len == 0:
            return 0

        pred_len = len(pred)
        ans_len = len(ans)
        return 2 / (pred_len / interaction_len + ans_len / interaction_len)


def eval_all(pred_list, ans_list):
    assert len(pred_list) == len(ans_list)
    return sum(eval(p, a) for p, a in zip(pred_list, ans_list)) / len(pred_list)

In [ ]:
"""
class mydata_nosep():
    def __init__(self,train_path,test_path):

        self.cc = OpenCC('t2s')
        train_data = pd.read_csv(train_path)
        test_data = pd.read_csv(test_path)
        # CONTENT / NAME / CKIPNAME
        train_x = [ self.clean_string(i) for i in train_data['full_content'].values.tolist()]
        train_y = [ self.name_list(i) for i in train_data['name'].values.tolist()]
        train_z = [ self.name_list(i) for i in train_data['ckip_names'].values.tolist() ]

        # CONTENT / NAME
        test_x = [self.clean_string(i) for i in test_data['full_content'].values.tolist()]
        self.test_y = [self.name_list(i) for i in test_data['name'].values.tolist()]
        #test_z = [self.name_list(i) for i in test_data['ckip_names'].values.tolist()]

        self.train = [] # CONTENT / NAME / (CKIP-NAME)
        self.test = []  # CONTENT / NAME

        
        qs = 0 
        for i in range(len(train_x)):
          if len(train_y[i]) == 0:
            #self.train.append([train_x[i],[],[]])
            continue
          q = []
          a = []
          for n in train_y[i]:
            qs += 1
            self.train.append([train_x[i],n,1])
          for n in train_z[i]-train_y[i]:
            if len(n)==1: # remove the ckip name len is 1
              continue
            qs += 1
            self.train.append([train_x[i],n,0])              

        for i in range(len(test_x)):
          self.test.append([test_x[i],self.test_y[i],])


        print(f'Train: {len(self.train)} Test: {len(self.test)} Questions: {qs}')

    def name_list(self,name_list_str):
        ls = ast.literal_eval(name_list_str)
        res = []
        cc = OpenCC('t2s')
        for n in ls:
          res.append(cc.convert(n))
        return set(res)        
    def clean_string(self,content):
      content = content.replace('\n','。').replace('\t','，').replace('!', '！').replace('?', '？')# erease white space cause English name error
      content = re.sub("[+\.\/_,$%●▼►^*(+\"\']+|[+——~@#￥%……&*（）★]", "",content)
      content = re.sub(r"[%s]+" %stops, "。",content)
      content = self.cc.convert(content)
      return content
"""

'\nclass mydata_nosep():\n    def __init__(self,train_path,test_path):\n\n        self.cc = OpenCC(\'t2s\')\n        train_data = pd.read_csv(train_path)\n        test_data = pd.read_csv(test_path)\n        # CONTENT / NAME / CKIPNAME\n        train_x = [ self.clean_string(i) for i in train_data[\'full_content\'].values.tolist()]\n        train_y = [ self.name_list(i) for i in train_data[\'name\'].values.tolist()]\n        train_z = [ self.name_list(i) for i in train_data[\'ckip_names\'].values.tolist() ]\n\n        # CONTENT / NAME\n        test_x = [self.clean_string(i) for i in test_data[\'full_content\'].values.tolist()]\n        self.test_y = [self.name_list(i) for i in test_data[\'name\'].values.tolist()]\n        #test_z = [self.name_list(i) for i in test_data[\'ckip_names\'].values.tolist()]\n\n        self.train = [] # CONTENT / NAME / (CKIP-NAME)\n        self.test = []  # CONTENT / NAME\n\n        \n        qs = 0 \n        for i in range(len(train_x)):\n          if len(tra

In [ ]:
class mydata_nosep():
    def __init__(self,train_path,test_path):

        self.cc = OpenCC('t2s')
        train_data = pd.read_csv(train_path)
        
        test_data = pd.read_csv(test_path)
        # CONTENT / NAME / CKIPNAME
        train_x = [ self.clean_string(i) for i in train_data['full_content'].values.tolist()]
        train_y = [ self.name_list(i) for i in train_data['name'].values.tolist()]
        train_z = [ self.name_list(i) for i in train_data['ckip_names'].values.tolist() ]

        # CONTENT / NAME
        test_x = [self.clean_string(i) for i in test_data['full_content'].values.tolist()]
        self.test_y = [self.name_list(i) for i in test_data['name'].values.tolist()]
        #test_z = [self.name_list(i) for i in test_data['ckip_names'].values.tolist()]

        self.train = [] # CONTENT / NAME / (CKIP-NAME)
        self.test = []  # CONTENT / NAME

        
        right,nright = 0,0
        for i in range(len(train_x)):
          if len(train_y[i]) == 0:
            #self.train.append([train_x[i],[],[]])
            continue
          q = []
          a = []
          for n in train_y[i]:
            right += 1
            self.train.append([train_x[i],n,1])

          for n in train_z[i]-train_y[i]:
            if len(n)==1: # remove the ckip name len is 1
              continue
            nright += 1
            self.train.append([train_x[i],n,0])              

        for i in range(len(test_x)):
          self.test.append([test_x[i],self.test_y[i],])


        print(f'Train: {len(self.train)} Test: {len(self.test)} R/N: {right},{nright}')

    def name_list(self,name_list_str):
        ls = ast.literal_eval(name_list_str)
        res = []
        cc = OpenCC('t2s')
        for n in ls:
          res.append(n)
        return set(res)        
    def clean_string(self,content):
      content = content.replace('\n','。').replace('\t','，').replace('!', '！').replace('?', '？').replace('；','。')# erease white space cause English name error
      content = re.sub("[+\.\/_,$%●▼►^*(+\"\']+|[+——~@#￥%……&*（）★]", "",content)
      content = re.sub(r"[%s]+" %stops, "。",content)
      content = content.replace(' ','')
      #content = self.cc.convert(content)
      return content

In [ ]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

class TrainDataset(Dataset):
    def __init__(self,data,model_type):
      self.data = data
      self.tokenizer = BertTokenizer.from_pretrained(model_type)
        
    def __getitem__(self,idx):
      question , paragraph ,label = self.data[idx][1] , self.data[idx][0],self.data[idx][2]
      token_tensor = self.tokenizer.encode_plus(question,paragraph,max_length=512,truncation=True,pad_to_max_length=True)
      label_tensor = torch.Tensor([label])
      # token / segment / mask / label
      return  torch.tensor(token_tensor['input_ids']), torch.tensor( token_tensor['token_type_ids']) , torch.tensor( token_tensor['attention_mask'] ),label_tensor
    def __len__(self):
      return len(self.data)


In [ ]:
class bertwwmQA(nn.Module):
    def __init__(self,model_name,config):
        super(bertwwmQA,self).__init__()
        #self.bert_model = BertModel.from_pretrained("hfl/chinese-roberta-wwm-ext-large")
        
        self.bert_model = BertModel.from_pretrained(model_name,config = config)
       # self.bert_model = BertForQuestionAnswering.from_pretrained(model_name,config = config)
        #self.bert_model = BertForSequenceClassification.from_pretrained(model_name,config = config)
        self.bi_decoder = nn.Sequential(
            nn.Linear(config.hidden_size,config.hidden_size)
            ,nn.Dropout(0.1)
            ,nn.ReLU()
            ,nn.Linear(config.hidden_size,2)
        )
        self.start()

    def start(self):
      nn.init.xavier_uniform_(self.bi_decoder[0].weight)
      nn.init.constant_(self.bi_decoder[0].bias, 0)
      nn.init.xavier_uniform_(self.bi_decoder[3].weight)
      nn.init.constant_(self.bi_decoder[3].bias, 0)

    def forward(self,input_ids=None,attention_mask=None,token_type_ids=None):
       # print(f'receive input_ids {input_ids}')

        cls = self.bert_model(input_ids,attention_mask=attention_mask,token_type_ids=token_type_ids)[1]
        binary = self.bi_decoder(cls)
        return binary

In [ ]:

x1 = pd.read_csv('./datasetALL/dataset1/tbrain_train.csv')
x2 = pd.read_csv('./datasetALL/dataset0/dataset0803.csv')
f = x1['full_content'].values.tolist()+x2['full_content'].values.tolist()
ck = x1['ckip_names'].values.tolist()+x2['ckip_names'].values.tolist()
na = x1['name'].values.tolist()+x2['name'].values.tolist()
import csv
with open('./datasetALL/dataset1/tbrain_train_ex.csv','w',encoding="utf-8",newline='') as csvfile:
  writer = csv.writer(csvfile)
  writer.writerow(['full_content','ckip_names','name'])
  for i in range(len(f)):
    writer.writerow([f[i],ck[i],na[i]])


# 打包啦

In [ ]:
"""
class QAFilter():
  def __init__(self,model_path): 
    
    # device
    self.device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
    
    # model & tokenizer
    model_type = 'hfl/chinese-roberta-wwm-ext'
    config = BertConfig.from_pretrained(model_type,output_hidden_states=True)
    self.tokenizer = BertTokenizer.from_pretrained(model_type)
    self.model = bertwwmQA(model_type,config).to(self.device)
    self.model.load_state_dict(torch.load(model_path)) 
  
    # 繁簡轉換
    self.c2tw = OpenCC('s2t') # china to tw
    self.tw2c = OpenCC('t2s') # tw to china

  def filter(self,content,name_list):
    # name_list : a list like: ['蔡英文','陳水扁']
    # content  : a new "繁體新聞"
    # 會回傳像是 [1,0,0,1] 代表第0和3的人名是對的
    content = self.tw2c.convert(content)
    pred_nlist = []
    with torch.no_grad():
      for n in name_list:
        n = self.tw2c.convert(n)
        token_tensor = self.tokenizer.encode_plus(str(n),str(content),max_length=512,truncation=True,pad_to_max_length=True)
        token = torch.tensor(token_tensor['input_ids']).unsqueeze(0).to(self.device)
        segment = torch.tensor( token_tensor['token_type_ids']).unsqueeze(0).to(self.device)
        mask = torch.tensor( token_tensor['attention_mask'] ).unsqueeze(0).to(self.device)
        pred = self.model(input_ids=token,attention_mask=mask,token_type_ids=segment).argmax(dim=1)
        #print(n,pred.item())
        pred_nlist.append(pred.item())
    return pred_nlist
"""

'\nclass QAFilter():\n  def __init__(self,model_path): \n    \n    # device\n    self.device = torch.device(\'cuda:0\') if torch.cuda.is_available() else torch.device(\'cpu\')\n    \n    # model & tokenizer\n    model_type = \'hfl/chinese-roberta-wwm-ext\'\n    config = BertConfig.from_pretrained(model_type,output_hidden_states=True)\n    self.tokenizer = BertTokenizer.from_pretrained(model_type)\n    self.model = bertwwmQA(model_type,config).to(self.device)\n    self.model.load_state_dict(torch.load(model_path)) \n  \n    # 繁簡轉換\n    self.c2tw = OpenCC(\'s2t\') # china to tw\n    self.tw2c = OpenCC(\'t2s\') # tw to china\n\n  def filter(self,content,name_list):\n    # name_list : a list like: [\'蔡英文\',\'陳水扁\']\n    # content  : a new "繁體新聞"\n    # 會回傳像是 [1,0,0,1] 代表第0和3的人名是對的\n    content = self.tw2c.convert(content)\n    pred_nlist = []\n    with torch.no_grad():\n      for n in name_list:\n        n = self.tw2c.convert(n)\n        token_tensor = self.tokenizer.encode_plus(str(n),str

In [ ]:

class QAFilter():
  def __init__(self,model_path): 
    
    # device
    self.device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
    
    # model & tokenizer
    model_type = 'hfl/chinese-roberta-wwm-ext'
    config = BertConfig.from_pretrained(model_type,output_hidden_states=True)
    self.tokenizer = BertTokenizer.from_pretrained(model_type)
    self.model = bertwwmQA(model_type,config).to(self.device)
    self.model.load_state_dict(torch.load(model_path)) 
  
    # 繁簡轉換
    self.c2tw = OpenCC('s2t') # china to tw
    self.tw2c = OpenCC('t2s') # tw to china

  def filter(self,content,name_list):
    # name_list : a list like: ['蔡英文','陳水扁']
    # content  : a new "繁體新聞"
    # 會回傳像是 [1,0,0,1] 代表第0和3的人名是對的
    content = self.tw2c.convert(content)
    pred_nlist = []
    with torch.no_grad():
      for n in name_list:
        n = self.tw2c.convert(n)
        token_tensor = self.tokenizer.encode_plus(str(n),str(content),max_length=512,truncation=True,pad_to_max_length=True)
        token = torch.tensor(token_tensor['input_ids']).unsqueeze(0).to(self.device)
        segment = torch.tensor( token_tensor['token_type_ids']).unsqueeze(0).to(self.device)
        mask = torch.tensor( token_tensor['attention_mask'] ).unsqueeze(0).to(self.device)
        pred = self.model(input_ids=token,attention_mask=mask,token_type_ids=segment).argmax(dim=1)
        #print(n,pred.item())
        pred_nlist.append(pred.item())
    return pred_nlist


In [ ]:
"""
def test(model,data,device,ws,pos,ner,model_name):
    #b_modelname = 'bertWWM_BI_16_27_53.pt'
    #bin_model = bertwwm().to(device)
    #bin_model.load_state_dict(torch.load(f'saved_models/{b_modelname}'))
    print('Testing...')
    test_num = len(data.test)
    #criterion = nn.BCELoss()
    #b_tokenizer = BertTokenizer.from_pretrained("hfl/chinese-bert-wwm")
    tokenizer = BertTokenizer.from_pretrained(model_name)
    model.eval()
    print(f'Testing data: {test_num}')
    c_to_tw = OpenCC('s2t') # chi to tw
    tw_to_c = OpenCC('t2s')
    label_count = 0
    all_predlist = []
    all_labellist = []
   # all_labellist = data.test_y
    with torch.no_grad():
      for content,nlabel in data.test:
        if len(nlabel) == 0:
         # all_labellist.append([])
          continue     
       # print(f'test name:')
        
        if len(content)>512:
          content = content[:512] 
        label_count += 1
        ckip_nlist = []
        pred_nlist = []
        ws_results = ws([c_to_tw.convert(content)])
        pos_results = pos(ws_results)
        ner_results = ner(ws_results, pos_results)
        for name in ner_results[0]:
          if name[2] == 'PERSON':
            ckip_nlist.append(tw_to_c.convert(name[3]))
        ckip_nlist = set(ckip_nlist)
        for ckip_name in ckip_nlist:
          if len(ckip_name)<2:
            continue
          token_tensor = tokenizer.encode_plus(str(ckip_name),str(content),max_length=512,truncation=True,pad_to_max_length=True)
          token = torch.tensor(token_tensor['input_ids']).unsqueeze(0).to(device)
          segment = torch.tensor( token_tensor['token_type_ids']).unsqueeze(0).to(device)
          mask = torch.tensor( token_tensor['attention_mask'] ).unsqueeze(0).to(device)
          pred = model(input_ids=token,attention_mask=mask,token_type_ids=segment).argmax(dim=1)
          if pred.item() == 1:
            pred_nlist.append(ckip_name)
       # pred_nlist = sorted(pred_nlist, key = lambda s: s[1])
        all_labellist.append(nlabel)
        all_predlist.append(pred_nlist)
        print(f'CKIP:{ckip_nlist} PRED:{pred_nlist} LABEL:{nlabel}')
        



    return all_predlist,all_labellist
"""

'\ndef test(model,data,device,ws,pos,ner,model_name):\n    #b_modelname = \'bertWWM_BI_16_27_53.pt\'\n    #bin_model = bertwwm().to(device)\n    #bin_model.load_state_dict(torch.load(f\'saved_models/{b_modelname}\'))\n    print(\'Testing...\')\n    test_num = len(data.test)\n    #criterion = nn.BCELoss()\n    #b_tokenizer = BertTokenizer.from_pretrained("hfl/chinese-bert-wwm")\n    tokenizer = BertTokenizer.from_pretrained(model_name)\n    model.eval()\n    print(f\'Testing data: {test_num}\')\n    c_to_tw = OpenCC(\'s2t\') # chi to tw\n    tw_to_c = OpenCC(\'t2s\')\n    label_count = 0\n    all_predlist = []\n    all_labellist = []\n   # all_labellist = data.test_y\n    with torch.no_grad():\n      for content,nlabel in data.test:\n        if len(nlabel) == 0:\n         # all_labellist.append([])\n          continue     \n       # print(f\'test name:\')\n        \n        if len(content)>512:\n          content = content[:512] \n        label_count += 1\n        ckip_nlist = []\n     

## 繁體測試

In [ ]:
def test(model,data,device,ws,pos,ner,model_name):
    print('Testing...')
    test_num = len(data.test)
    tokenizer = BertTokenizer.from_pretrained(model_name)
    model.eval()
    print(f'Testing data: {test_num}')
    label_count = 0
    all_predlist = []
    all_labellist = []
    with torch.no_grad():
      for content,nlabel in data.test:
        if len(nlabel) == 0:
          continue           
        #if len(content)>512:
        #  content = content[:512] 
        label_count += 1
        ckip_nlist = []
        pred_nlist = []
        ws_results = ws([content])
        pos_results = pos(ws_results)
        ner_results = ner(ws_results, pos_results)
        for name in ner_results[0]:
          if name[2] == 'PERSON':
            ckip_nlist.append(name[3])
        ckip_nlist = set(ckip_nlist)
        for ckip_name in ckip_nlist:
          if len(ckip_name)<2:
            continue
          token_tensor = tokenizer.encode_plus(str(ckip_name),str(content),max_length=512,truncation=True,pad_to_max_length=True)
          token = torch.tensor(token_tensor['input_ids']).unsqueeze(0).to(device)
          segment = torch.tensor( token_tensor['token_type_ids']).unsqueeze(0).to(device)
          mask = torch.tensor( token_tensor['attention_mask'] ).unsqueeze(0).to(device)
          pred = model(input_ids=token,attention_mask=mask,token_type_ids=segment).argmax(dim=1)
          if pred.item() == 1:
            pred_nlist.append(ckip_name)
       # pred_nlist = sorted(pred_nlist, key = lambda s: s[1])
        all_labellist.append(nlabel)
        all_predlist.append(pred_nlist)
        print(f'CKIP:{ckip_nlist} PRED:{pred_nlist} LABEL:{nlabel}')

    return all_predlist,all_labellist

In [ ]:
def testBIN(model,data,device,ws,pos,ner,model_name):
    print('Testing...')
    acc = 0
    total = len(data.test)
    tokenizer = BertTokenizer.from_pretrained(model_name)
    with torch.no_grad():
      for content,name in data.test:
        input_ids = torch.tensor([tokenizer.encode(str(content),max_length=512,truncation=True,pad_to_max_length=True)]).to(device)   
        b_pred = model(input_ids).to(device) 
        if b_pred.item() < 0.4 and len(name) == 0:
          acc +=1
        elif b_pred.item() > 0.4 and len(name):
          acc +=1
        print(f'C: {content[:20]} Name: {name} Pred: {b_pred.item()}')
    print(f'ALL {total} ACC {acc}')

In [ ]:
def train(trainLoader,w_d,lr_rate,device,model_type,epoches):
  config = BertConfig.from_pretrained(model_type, output_hidden_states=True)
  model = bertwwmQA(model_type,config).to(device)
  parameters = filter(lambda p: p.requires_grad, model.parameters())
  optimizer = AdamW(parameters, lr=lr_rate, weight_decay=w_d)
  #criterion = nn.BCELoss()
  criterion = nn.CrossEntropyLoss()
  model.train()
  minloss = 100000
  for e in range(epoches):
    loss,i = 0,0
    for data in trainLoader:
      i+=1
      token , segment , mask , label = [ item.to(device) for item in data]
      pred = torch.softmax(model(input_ids=token,attention_mask=mask,token_type_ids=segment),dim=-1)   
      label = label.reshape(label.shape[0])
     # print(pred.shape,label.shape)
      batch_loss = criterion(pred,label.long())
      loss += batch_loss
     # print(batch_loss)
      batch_loss.backward()
     # print(f'Epoch {e} Batch {i} Loss is {batch_loss}')
      optimizer.step()
      optimizer.zero_grad()
      #print(f'Loss {batch_loss}')
    print(f'Epoches: {e} Loss {loss} AVG: {loss/i}')

    if loss < minloss:
      minloss = loss
      best_model = copy.deepcopy(model.state_dict())
  return best_model

In [ ]:
#model_type = 'hfl/chinese-bert-wwm'


In [ ]:
#mydata = mydata_nosep('./data0/tbrain_train.csv','./data0/tbrain_test.csv')

In [ ]:

!pip install -U ckiptagger[tfgpu,gdown]
from ckiptagger import WS, POS, NER
ws = WS("./data")
pos = POS("./data")
ner = NER("./data")
print('CKIP finish')


     |████████████████████████████████| 411.0MB 39kB/s 
     |████████████████████████████████| 512kB 43.9MB/s 
     |████████████████████████████████| 51kB 8.4MB/s 
     |████████████████████████████████| 3.8MB 48.7MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=a61e9556738849714d86262b0527a517386cf16e2ef0efd6f16ecb0a9064cee4
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow 2.3.0 has requirement gast==0.3.3, but you'll have gast 0.2.2 which is incompatible.
ERROR: tensorflow 2.3.0 has requirement tensorboard<3,>=2.3.0, but you'll have tensorboard 1.15.0 which is incompatible.
ERROR: tensorflow 2.3.0 has requirement tensorflow-estimator<2.4.0,>=2.3.0, but you'll have tensorflow-estimator 1.15.1 which is incompatible.
ERROR: tensorflow-probability 0.11.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing i

AttributeError: ignored

In [ ]:
"""
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
for i in range(1,11):
  print(f'Train QA in dataset {i}')
  tpath = './datasetALL/dataset'+ str(i) + '/tbrain_train.csv'
  print(tpath)
  mydata = mydata_nosep(tpath,'./data0/tbrain_test.csv')
  model_type = 'hfl/chinese-roberta-wwm-ext'
  train_ds = TrainDataset(mydata.train,model_type)
  trainLoader = DataLoader(train_ds, batch_size=8)
  config = BertConfig.from_pretrained(model_type,output_hidden_states=True)
  best_model = train(trainLoader,0.001,1e-5,device,model_type,10)
  if not os.path.exists('saved_models'):
    os.makedirs('saved_models')    
  
  modelname = 'bertWWM_QA'+ str(i) + '_'
  torch.save(best_model, f'saved_models/{modelname}.pt')
  print(f'Train end, model name is {modelname}.pt')
"""

In [ ]:
mydata = mydata_nosep('./datasetALL/dataset1/tbrain_train_ex.csv','./datasetALL/dataset1/tbrain_test.csv')

In [ ]:
mydata.train[:10]
mydata.test[:10]

In [ ]:
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')

model_type = 'hfl/chinese-roberta-wwm-ext'
torch.cuda.empty_cache()  


In [ ]:
mode = 'testqa'

In [ ]:
torch.cuda.empty_cache() 
if mode == 'trainqa':
  model_type = "bert-base-chinese"
  #model_type = 'hfl/chinese-bert-wwm'
  #model_type = 'hfl/chinese-roberta-wwm-ext-large'
  train_ds = TrainDataset(mydata.train,model_type)
  trainLoader = DataLoader(train_ds, batch_size=8)
  config = BertConfig.from_pretrained(model_type,output_hidden_states=True)
  print('Train QA')
  best_model = train(trainLoader,0.001,1e-5,device,model_type,10)
  if not os.path.exists('saved_models'):
    os.makedirs('saved_models')    
  modeltime = strftime('%H_%M_%S', gmtime()) 
  modelname = 'bertWWM_QA_'+ modeltime
  torch.save(best_model, f'saved_models/{modelname}.pt')
  print(f'Train end, model name is {modelname}.pt')

elif mode == 'testqa':
  print('Testqa')
  #model_type = 'hfl/chinese-roberta-wwm-ext'
  model_type = "bert-base-chinese"
  modelname = 'bertWWM_QA_09_44_57.pt'
  config = BertConfig.from_pretrained(model_type,output_hidden_states=True)

  test_model = bertwwmQA(model_type,config).to(device)
  test_model.load_state_dict(torch.load(f'saved_models/{modelname}'))
  all_predlist,all_labellist = test(test_model,mydata,device,ws,pos,ner,model_type)
  print(f'Testend point is {eval_all(all_predlist,all_labellist)}')
# train binary classfier
elif mode == 'trainbin':
  model_type = 'hfl/chinese-bert-wwm'
  best_model = trainBinary(data,0.005,5e-4,device,model_type)
  if not os.path.exists('saved_models'):
    os.makedirs('saved_models')    
  modeltime = strftime('%H_%M_%S', gmtime()) 
  modelname = 'bertWWM_BIN_'+ modeltime
  torch.save(best_model, f'saved_models/{modelname}.pt')
  print(f'Train end, model name is {modelname}.pt')

elif mode == 'testbin':
  modelname = 'bertWWM_BIN_15_05_01.pt'
  model_type = 'hfl/chinese-bert-wwm'
 # test_model = bertwwm(model_type).to(device)
  test_model = bertwwm(model_type).to(device)
  test_model.load_state_dict(torch.load(f'saved_models/{modelname}'))
  
  testBIN(test_model,data,device,ws,pos,ner,model_type)

In [ ]:
print(f'Testend point is {eval_all(all_predlist,all_labellist)}')

# bert-base-chinese 1e-5 : 0.88 

In [ ]:
print(f'Testend point is {eval_all(all_predlist,all_labellist)}')

# bert-base-chinese 1e-5 : 0.88 

In [ ]:
#RoBERTa-wwm-ext-large	hfl/chinese-roberta-wwm-ext-large
#RoBERTa-wwm-ext	hfl/chinese-roberta-wwm-ext
#BERT-wwm-ext	hfl/chinese-bert-wwm-ext
#BERT-wwm	hfl/chinese-bert-wwm
#RBT3	hfl/rbt3
#RBTL3	hfl/rbtl3